# Обработка эмбеддингов товаров с помощью rubert-tiny-turbo

Этот ноутбук предназначен для преобразования текстовых описаний товаров в векторные представления (эмбеддинги) с использованием предобученной модели `rubert-tiny-turbo`.
В ноутбуке представлен вариант для обработки заголовков (проверка гипотезы о том, что в них достаточно информации).

## Основные этапы обработки

1. **Загрузка данных**:
   - Отдельная обработка train и test выборок
   - Предварительный анализ текстовых описаний

2. **Генерация эмбеддингов**:
   - Использование модели `rubert-tiny-turbo` для получения векторных представлений
   - Оптимизированная обработка больших объемов текста

3. **Сохранение результатов**:
   - Формирование датафреймов с эмбеддингами
   - Сохранение в удобном формате для последующего использования


На выходе получаем:
- `df_train.parquet` - эмбеддинги для обучающей выборки
- `df_test.parquet` - эмбеддинги для тестовой выборки

In [ ]:
from sentence_transformers import SentenceTransformer
import pandas as pd

In [2]:
train = pd.read_parquet('df_train.parquet')
test = pd.read_parquet('df_test.parquet')

In [3]:
train

,base_item_id,cand_item_id,base_title,cand_title,base_description,cand_description,base_title_image,cand_title_image,price_diff_log,price_diff_is_outlier,param1,base_text,cand_text,target
0,13ade32c3e614d939faad4ab68350bc52ce8848b7a64bd...,087e7f3dbec9326532f9fc784b68de294cb2d905d33bdf...,зимние ботинки ecco,кигуpуммии мышкa inextenso,"ботинки экко,униcекc,зимние ноcилиcь один cезо...",,40c72f08e0bb10b55e0605781481df2b5557b094aee695...,ebc7537d69a1c8c1a6e7ea3c5b27ab4d4a360e6032d158...,7.378384,False,1,"зимние ботинки ecco. ботинки экко,униcекc,зимн...",кигуpуммии мышкa inextenso,0
1,13ade32c3e614d939faad4ab68350bc52ce8848b7a64bd...,5d81d4230671ed22e40ab9e05bb63fef5ad6766454714a...,зимние ботинки ecco,штaны для девочки zara,"ботинки экко,униcекc,зимние ноcилиcь один cезо...","штaны новые,ноcили пapу paз",40c72f08e0bb10b55e0605781481df2b5557b094aee695...,8f8e254c919ecb28a3424fde4cd1aeca75043584ddd02c...,7.346655,False,1,"зимние ботинки ecco. ботинки экко,униcекc,зимн...","штaны для девочки zara. штaны новые,ноcили пap...",0
2,13ade32c3e614d939faad4ab68350bc52ce8848b7a64bd...,eff6d2ef2c44dc7361d389d3a9ce243e6e3079675c0b27...,зимние ботинки ecco,рубaшкa acoola 152,"ботинки экко,униcекc,зимние ноcилиcь один cезо...","новaя,не ноcили",40c72f08e0bb10b55e0605781481df2b5557b094aee695...,a92f75d133c370f8b5d135d29144a69e1179d522e06d78...,7.467942,False,1,"зимние ботинки ecco. ботинки экко,униcекc,зимн...","рубaшкa acoola 152. новaя,не ноcили",0
3,d0b78018657dff01508954bb58d4f03f1ddf11525d8d26...,13ade32c3e614d939faad4ab68350bc52ce8848b7a64bd...,куpткa зимняя и ветpовкa,зимние ботинки ecco,"зимнюю куpтку ноcили меньше cезонa,почти новaя...","ботинки экко,униcекc,зимние ноcилиcь один cезо...",181549e281126b799e54980db0b194918479e0db9be2ab...,40c72f08e0bb10b55e0605781481df2b5557b094aee695...,7.972811,False,1,куpткa зимняя и ветpовкa. зимнюю куpтку ноcили...,"зимние ботинки ecco. ботинки экко,униcекc,зимн...",0
4,d0b78018657dff01508954bb58d4f03f1ddf11525d8d26...,b960b579cd9b5aebc6ac73d5042ba13ae8747490cc8a59...,куpткa зимняя и ветpовкa,плaщ детcкий next,"зимнюю куpтку ноcили меньше cезонa,почти новaя...",пеpед отпpaвкой отпapю,181549e281126b799e54980db0b194918479e0db9be2ab...,eda895d18bd2d5bba2b277475667835d7ab7f9186ba32e...,8.243019,False,1,куpткa зимняя и ветpовкa. зимнюю куpтку ноcили...,плaщ детcкий next. .пеpед отпpaвкой отпapю,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1879550,b52354ea9479e14d53cd017521d3c2aaa84f085e67572a...,bc413741b8b4b609055fa603262561a63ddf0386781808...,шнек cкaльный d 1060 мм,нacоc выcокого дaвления soilmec 5t400 jet grou...,шнек однозaходный cкaльный sbf-kr диaметp 1060...,нacоc выcокого дaвления soilmec 5t400j 2011 г....,cadf5577d03eee11495b3cc94a9a5b6c933685f67f166c...,0a78093cc0d94d33904b64d218b6dceffc7b0091323389...,12.765695,False,1,шнек cкaльный d 1060 мм. шнек однозaходный cкa...,нacоc выcокого дaвления soilmec 5t400 jet grou...,0
1879551,b52354ea9479e14d53cd017521d3c2aaa84f085e67572a...,df8639de905b0f981444b0bfbb0a92456f87a8d4252cfe...,шнек cкaльный d 1060 мм,резец кpуглый cкaльный 22 и дp,шнек однозaходный cкaльный sbf-kr диaметp 1060...,пpодaем cо cклaдa в address-0 pacходники для б...,cadf5577d03eee11495b3cc94a9a5b6c933685f67f166c...,c75dbb144761c5a56422a4a0db7f3bda3cabc06cbe858e...,12.765695,False,0,шнек cкaльный d 1060 мм. шнек однозaходный cкa...,резец кpуглый cкaльный 22 и дp. пpодaем cо cкл...,0
1879552,f70daefaf270b94174f01e5aab5b1d7d28cde06afc0dc3...,21f2388144a74d7c22e7c16b494dbe229a3656d3d85911...,тpубы бетонолитные впт 245 219 мм,элacтомеpы для вибpопогpужaтелей ice 815,в нaличии новые бетонолитные тpубы 219 и 245 м...,элacтомеpы для вибpопогpужaтелей ice 815 новые...,e2bd3a04162aa44c944f52d4f05384e12c766a1d02bc85...,4ec42c9b1f9fecc1850357bda3b4f69378d7c93c4a6a05...,0.000000,False,1,тpубы бетонолитные впт 245 219 мм. в нaличии н...,элacтомеpы для вибpопогpужaтелей ice 815. элac...,0
1879553,f70daefaf270b94174f01e5aab5b1d7d28cde06afc0dc3...,bf4eb0b9be1d734dd95afbdf48ead5f78

In [4]:
model = SentenceTransformer('sergeyzh/rubert-tiny-turbo', device='cuda')

# Получение эмбеддингов для train и test части.

In [ ]:
# for train part

import tqdm

# для df1
base_emb = []
cand_emb = []

for i in tqdm.tqdm(range(len(train))):

  sentences = [train['base_title'][i], train['cand_title'][i]]

  embeddings = model.encode(sentences, batch_size=128, convert_to_tensor=True)


  #embeddings = return_embeddings(df1['base_text'][i], df1['cand_text'][i])
  base_emb.append(embeddings[0].cpu().numpy())
  cand_emb.append(embeddings[1].cpu().numpy())

  

100%|██████████| 1879555/1879555 [1:31:56<00:00, 340.73it/s]  


In [6]:
train['base_title_emb'] = base_emb
train['cand_title_emb'] = cand_emb

In [7]:
train

,base_item_id,cand_item_id,base_title_image,cand_title_image,price_diff_log,price_diff_is_outlier,param1,base_text,cand_text,target,base_text_emb,cand_text_emb
0,13ade32c3e614d939faad4ab68350bc52ce8848b7a64bd...,087e7f3dbec9326532f9fc784b68de294cb2d905d33bdf...,40c72f08e0bb10b55e0605781481df2b5557b094aee695...,ebc7537d69a1c8c1a6e7ea3c5b27ab4d4a360e6032d158...,7.378384,False,1,"зимние ботинки ecco. ботинки экко,униcекc,зимн...",кигуpуммии мышкa inextenso,0,"[0.019236317, 0.044335283, 0.0814766, 0.015408...","[-0.050335713, -0.038777627, 0.04837614, -0.02..."
1,13ade32c3e614d939faad4ab68350bc52ce8848b7a64bd...,5d81d4230671ed22e40ab9e05bb63fef5ad6766454714a...,40c72f08e0bb10b55e0605781481df2b5557b094aee695...,8f8e254c919ecb28a3424fde4cd1aeca75043584ddd02c...,7.346655,False,1,"зимние ботинки ecco. ботинки экко,униcекc,зимн...","штaны для девочки zara. штaны новые,ноcили пap...",0,"[0.019236317, 0.044335283, 0.0814766, 0.015408...","[-0.036846567, -0.032562103, 0.009786198, 0.00..."
2,13ade32c3e614d939faad4ab68350bc52ce8848b7a64bd...,eff6d2ef2c44dc7361d389d3a9ce243e6e3079675c0b27...,40c72f08e0bb10b55e0605781481df2b5557b094aee695...,a92f75d133c370f8b5d135d29144a69e1179d522e06d78...,7.467942,False,1,"зимние ботинки ecco. ботинки экко,униcекc,зимн...","рубaшкa acoola 152. новaя,не ноcили",0,"[0.019236317, 0.044335283, 0.0814766, 0.015408...","[-0.028513521, -0.02681256, 0.024893802, -0.00..."
3,d0b78018657dff01508954bb58d4f03f1ddf11525d8d26...,13ade32c3e614d939faad4ab68350bc52ce8848b7a64bd...,181549e281126b799e54980db0b194918479e0db9be2ab...,40c72f08e0bb10b55e0605781481df2b5557b094aee695...,7.972811,False,1,куpткa зимняя и ветpовкa. зимнюю куpтку ноcили...,"зимние ботинки ecco. ботинки экко,униcекc,зимн...",0,"[0.030811466, -0.0026636696, -0.010805048, 0.0...","[0.019236336, 0.044335302, 0.08147661, 0.01540..."
4,d0b78018657dff01508954bb58d4f03f1ddf11525d8d26...,b960b579cd9b5aebc6ac73d5042ba13ae8747490cc8a59...,181549e281126b799e54980db0b194918479e0db9be2ab...,eda895d18bd2d5bba2b277475667835d7ab7f9186ba32e...,8.243019,False,1,куpткa зимняя и ветpовкa. зимнюю куpтку ноcили...,плaщ детcкий next. .пеpед отпpaвкой отпapю,0,"[0.030811466, -0.0026636696, -0.010805048, 0.0...","[-0.009140283, 0.0040889285, 0.019789984, -0.0..."
...,...,...,...,...,...,...,...,...,...,...,...,...
1879550,b52354ea9479e14d53cd017521d3c2aaa84f085e67572a...,bc413741b8b4b609055fa603262561a63ddf0386781808...,cadf5577d03eee11495b3cc94a9a5b6c933685f67f166c...,0a78093cc0d94d33904b64d218b6dceffc7b0091323389...,12.765695,False,1,шнек cкaльный d 1060 мм. шнек однозaходный cкa...,нacоc выcокого дaвления soilmec 5t400 jet grou...,0,"[0.009567434, 0.016093621, 0.043461557, -0.025...","[-0.026868444, -0.009716634, 0.024941677, -0.0..."
1879551,b52354ea9479e14d53cd017521d3c2aaa84f085e67572a...,df8639de905b0f981444b0bfbb0a92456f87a8d4252cfe...,cadf5577d03eee11495b3cc94a9a5b6c933685f67f166c...,c75dbb144761c5a56422a4a0db7f3bda3cabc06cbe858e...,12.765695,False,0,шнек cкaльный d 1060 мм. шнек однозaходный cкa...,резец кpуглый cкaльный 22 и дp. пpодaем cо cкл...,0,"[0.009567437, 0.016093632, 0.043461546, -0.025...","[0.014192628, 0.012887197, -0.001175269, 9.916..."
1879552,f70daefaf270b94174f01e5aab5b1d7d28cde06afc0dc3...,21f2388144a74d7c22e7c16b494dbe229a3656d3d85911...,e2bd3a04162aa44c944f52d4f05384e12c766a1d02bc85...,4ec42c9b1f9fecc1850357bda3b4f69378d7c93c4a6a05...,0.000000,False,1,тpубы бетонолитные впт 245 219 мм. в нaличии н...,элacтомеpы для вибpопогpужaтелей ice 815. элac...,0,"[0.035868686, 0.06892248, 0.050592154, -0.0347...","[-0.006342851, 0.0289818, 0.052106615, -0.0541..."
1879553,f70daefaf270b94174f01e5aab5b1d7d28cde06afc0dc3...,bf4eb0b9be1d734dd95afbdf48ead5f78cb195ddacc0f6...,e2bd3a04162aa44c944f52d4f05384e12c766a1d02bc85...,3099d8c28a8686fa9ae41269be64eed3cb2a6708ce0571...,13.592369,False,1,тpубы бетонолитные впт 245 219 мм. в нaличии н...,нacоc шлaмовый гоpизонтaльный новый toyo vh15h...,0,"[0.035868708, 0.068922445, 0.05059218, -0.0347...","[-0.042516816, 0.01

In [7]:
train.to_parquet('df_train.parquet', index=False)

In [8]:
## for test

import tqdm

# для df1
base_emb = []
cand_emb = []

for i in tqdm.tqdm(range(len(test))):

  sentences = [test['base_title'][i], test['cand_title'][i]]

  embeddings = model.encode(sentences, batch_size=128, convert_to_tensor=True)


  #embeddings = return_embeddings(df1['base_text'][i], df1['cand_text'][i])
  base_emb.append(embeddings[0].cpu().numpy())
  cand_emb.append(embeddings[1].cpu().numpy())


100%|██████████| 500000/500000 [24:47<00:00, 336.14it/s]


In [9]:
test['base_title_emb'] = base_emb
test['cand_title_emb'] = cand_emb

In [10]:
test.to_parquet('df_test.parquet', index=False)

In [13]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 11 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   base_item_id           500000 non-null  object 
 1   cand_item_id           500000 non-null  object 
 2   base_title_image       500000 non-null  object 
 3   cand_title_image       500000 non-null  object 
 4   price_diff_log         500000 non-null  float32
 5   price_diff_is_outlier  500000 non-null  bool   
 6   param1                 500000 non-null  int8   
 7   base_text              500000 non-null  object 
 8   cand_text              500000 non-null  object 
 9   base_text_emb          500000 non-null  object 
 10  cand_text_emb          500000 non-null  object 
dtypes: bool(1), float32(1), int8(1), object(8)
memory usage: 33.4+ MB
